### Feature Engineering: Encoding of Categorical features using Neural Networks embeddings
#### Code adapted from Shivanand Roy's github https://github.com/Shivanandroy

Only works for supervised datasets. 
Datasets: HR_Attrition


In [22]:
from keras.models import Sequential, Model, model_from_json
from keras.layers import Dense, Dropout, Embedding, Activation, Input, concatenate, Reshape, Flatten
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import PReLU
from keras.optimizers import Adam
from keras_tqdm import TQDMNotebookCallback
from keras import backend as K
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.validation import check_is_fitted
from sklearn.utils import column_or_1d
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import warnings
from tqdm import tqdm_notebook
warnings.filterwarnings("ignore")

In [23]:
# Reading data
df = pd.read_csv("https://academic.uprm.edu/eacuna/HR_Attrition_Data.csv")
df.shape

(54808, 14)

In [24]:
df.head()

,employee_id,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,is_promoted
0,65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,5.0,8,1,0,49,0
1,65141,Operations,region_22,Bachelor's,m,other,1,30,5.0,4,0,0,60,0
2,7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,3.0,7,0,0,50,0
3,2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,1.0,10,0,0,50,0
4,48945,Technology,region_26,Bachelor's,m,other,1,45,3.0,2,0,0,73,0


In [25]:
#Definiendo las predictoras y las clases
X = df.drop(['employee_id', 'is_promoted'], axis=1)
y = df['is_promoted']
X.shape
print(type(y))

<class 'pandas.core.series.Series'>


In [26]:
def get_embedding_info(data, categorical_variables=None):
    '''
    this function identifies categorical variables and its embedding size
    :data: input data [dataframe]
    :categorical_variables: list of categorical_variables [default: None]
    if None, it automatically takes the variables with data type 'object'
    embedding size of categorical variables are determined by minimum of 50 or half of the no. of its unique values.
    i.e. embedding size of a column  = Min(50, # unique values of that column)
    '''
    if categorical_variables is None:
        categorical_variables = data.select_dtypes(include='object').columns

    return {col:(data[col].nunique(),min(50,(data[col].nunique()+ 1) //2)) for col in categorical_variables}


In [27]:
# get_embedding_info identifies the categorical variables and the number 
#of unique values and embedding size (half of unique values) and returns a dictionary
embedding_info = get_embedding_info(X)
embedding_info

{'department': (9, 5),
 'region': (34, 17),
 'education': (3, 2),
 'gender': (2, 1),
 'recruitment_channel': (3, 2)}

In [28]:
# Helper functions:

class __LabelEncoder__(LabelEncoder):

    def transform(self, y):

        check_is_fitted(self, 'classes_')
        y = column_or_1d(y, warn=True)

        unseen = len(self.classes_)

        e = np.array([
                     np.searchsorted(self.classes_, x)
                     if x in self.classes_ else unseen
                     for x in y
                     ])

        if unseen in e:
            self.classes_ = np.array(self.classes_.tolist() + ['unseen'])

        return e



In [29]:
def get_label_encoded_data(data, categorical_variables=None):
    '''
    this function label encodes all the categorical variables using sklearn.preprocessing.labelencoder
    and returns a label encoded dataframe for training
    :data: input data [dataframe]
    :categorical_variables: list of categorical_variables [Default: None]
    if None, it automatically takes the variables with data type 'object'
    '''
    encoders = {}

    df = data.copy()

    if categorical_variables is None:
        categorical_variables = [col for col in df.columns if df[col].dtype == 'object']

    for var in categorical_variables:
        #print(var)
        encoders[var] = __LabelEncoder__()
        df.loc[:, var] = encoders[var].fit_transform(df[var])

    return df, encoders

In [30]:
# get_label_encoded_data integer encodes the categorical variables and prepares it to feed it to neural network
X_encoded,encoders = get_label_encoded_data(X)
X_encoded.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
0,7,31,2,0,2,1,35,5.0,8,1,0,49
1,4,14,0,1,0,1,30,5.0,4,0,0,60
2,7,10,0,1,2,1,34,3.0,7,0,0,50
3,7,15,0,1,0,2,39,1.0,10,0,0,50
4,8,18,0,1,0,1,45,3.0,2,0,0,73


In [31]:
# Main function:

def get_embeddings(X_train, y_train, categorical_embedding_info, epochs=100, batch_size=256):
    '''
    this function trains a shallow neural networks and returns embeddings of categorical variables
    :X_train: training data [dataframe]
    :y_train: target variable
    :categorical_embedding_info: output of get_embedding_info function [dictionary of categorical variable and it's embedding size]
    :is_classification: True for classification tasks; False for regression tasks
    :epochs: num of epochs to train [default:100]
    :batch_size: batch size to train [default:256]
    It is a 2 layer neural network architecture with 1000 and 500 neurons with 'ReLU' activation
    for classification: loss = 'binary_crossentropy'; metrics = 'accuracy'
    for regression: loss = 'mean_squared_error'; metrics = 'r2'
    '''

    numerical_variables = [x for x in X_train.columns if x not in list(categorical_embedding_info.keys())]

    inputs = []
    flatten_layers = []

    for var, sz in categorical_embedding_info.items():
        input_c = Input(shape=(1,), dtype='int32')
        embed_c = Embedding(*sz, input_length=1)(input_c)
        flatten_c = Flatten()(embed_c)
        inputs.append(input_c)
        flatten_layers.append(flatten_c)
        #print(inputs)

    input_num = Input(shape=(len(numerical_variables),), dtype='float32')
    flatten_layers.append(input_num)
    inputs.append(input_num)

    flatten = concatenate(flatten_layers, axis=-1)

    fc1 = Dense(1000, kernel_initializer='normal')(flatten)
    fc1 = Activation('relu')(fc1)



    fc2 = Dense(500, kernel_initializer='normal')(fc1)
    fc2 = Activation('relu')(fc2)

    output = Dense(1, activation='sigmoid')(fc2)


    nnet = Model(inputs=inputs, outputs=output)

    x_inputs = []
    for col in categorical_embedding_info.keys():
        x_inputs.append(X_train[col].values)

    x_inputs.append(X_train[numerical_variables].values)


    nnet.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    nnet.fit(x_inputs, y_train.values, batch_size=batch_size, epochs=epochs, validation_split=0.2, verbose=0)

    embs = list(map(lambda x: x.get_weights()[0], [x for x in nnet.layers if 'Embedding' in str(x)]))
    embeddings = {var: emb for var, emb in zip(categorical_embedding_info.keys(), embs)}
    return embeddings


In [32]:
def get_embeddings_in_dataframe(embeddings, encoders):
    '''
    this function return the embeddings in pandas dataframe
    :embeddings: output of 'get_embeddings' function
    :encoders: output of 'get_embedding_info' function
    '''

    assert len(embeddings)==len(encoders), "Categorical variables in embeddings does not match with those of encoders"

    dfs={}
    for cat_var in tqdm_notebook(embeddings.keys()):
        df = pd.DataFrame(embeddings[cat_var])
        df.index = encoders[cat_var].classes_
        df.columns = [cat_var +  '_embedding_' + str(num) for num in df.columns]
        dfs[cat_var] = df

    return dfs


def fit_transform(data, embeddings, encoders, drop_categorical_vars=False):
    '''
    this function includes the trained embeddings into your data
    :data: input data [dataframe]
    :embeddings: output of 'get_embeddings' function
    :encoders: output of 'get_embedding_info' function
    :drop_categorical_vars: False to keep the categorical variables in the data along with the embeddings
    if True - drops the categorical variables and replaces them with trained embeddings
    '''

    assert len(embeddings)==len(encoders), "Categorical variables in embeddings does not match with those of encoders"

    dfs={}
    for cat_var in tqdm_notebook(embeddings.keys()):
        df = pd.DataFrame(embeddings[cat_var])
        df.index = encoders[cat_var].classes_
        df.columns = [cat_var +  '_embedding_' + str(num) for num in df.columns]
        data = data.merge(df, how='left', left_on=cat_var, right_index=True)

    if drop_categorical_vars:
        return data.drop(list(embeddings.keys()), axis=1)
    else:
        return data

In [33]:
# splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X_encoded,y)

# ce.get_embeddings trains NN, extracts embeddings and return a dictionary containing the embeddings
embeddings = get_embeddings(X_train, y_train, categorical_embedding_info=embedding_info,  epochs=100,batch_size=256)

In [34]:
# if you don't like the dictionary format; convert it to dataframe for easy readibility
dfs = get_embeddings_in_dataframe(embeddings=embeddings, encoders=encoders)

In [35]:
dfs["region"]

,region_embedding_0,region_embedding_1,region_embedding_2,region_embedding_3,region_embedding_4,region_embedding_5,region_embedding_6,region_embedding_7,region_embedding_8,region_embedding_9,region_embedding_10,region_embedding_11,region_embedding_12,region_embedding_13,region_embedding_14,region_embedding_15,region_embedding_16
region_1,0.301105,-0.253629,0.153161,-0.143966,0.112713,-0.178458,-0.136050,0.491469,0.160871,0.148288,-0.371751,0.169066,-0.002237,-0.225597,-0.139695,-0.149796,-0.101879
region_10,0.171763,-0.108698,0.179423,-0.112855,0.209821,-0.195363,-0.190488,-0.081082,0.198763,0.097487,-0.027178,0.090814,-0.261242,-0.166418,-0.150817,-0.238339,-0.223826
region_11,0.178534,-0.200949,0.189087,-0.189298,0.186133,-0.202308,-0.113259,0.435566,0.093707,0.256732,-0.228189,0.140014,-0.210803,-0.146883,-0.173900,-0.186005,-0.141134
region_12,0.195210,-0.158056,0.245482,-0.164145,0.215089,-0.111196,-0.087027,0.381301,0.156775,0.269878,-0.045881,0.142849,-0.250162,-0.104608,-0.245356,-0.206457,-0.182183
region_13,0.154489,-0.201254,0.151774,-0.157379,0.104692,-0.170736,-0.146646,0.206570,0.250468,0.159473,-0.171303,0.177124,-0.148150,-0.141073,-0.232942,-0.203853,-0.162361
region_14,0.273302,-0.147915,0.150003,-0.189628,0.146271,-0.203503,-0.099694,0.365680,0.223782,0.165970,-0.267615,0.150591,-0.107356,-0.193301,-0.130029,-0.153587,-0.171024
region_15,0.123456,-0.191489,0.170655,-0.174063,0.125578,-0.207405,-0.195210,-0.073354,0.173398,0.163155,-0.140850,0.191158,-0.092797,-0.163895,-0.198719,-0.145966,-0.185650
region_16,0.113894,-0.217684,0.220480,-0.178193,0.130641,-0.118714,-0.117686,0.078907,0.133295,0.192153,-0.141698,0.144560,-0.246932,-0.158387,-0.189572,-0.195021,-0.195348
region_17,0.068529,-0.170144,0.101861,-0.127422,0.198091,-0.037312,-0.131640,-0.459297,0.191534,0.146755,-0.247444,0.187294,-0.137707,-0.152378,-0.185872,-0.172718,-0.243368
region_18,0.380826,-0.328211,0.316902,-0.382539,0.300163,-0.390388,-0.362626,-0.246996,0.418707,0.266815,-0.343536,0.347021,-0.338535,-0.420311,-0.306112,-0.378708,-0.253079


In [36]:
#datos=pd.read_csv("http://academic.uprm.edu/eacuna/heartc.csv",sep=",",na_values=["?"])
datos=pd.read_csv("http://academic.uprm.edu/eacuna/heartc1.csv",sep=",")
#datos = pd.read_csv('c://PW-PR/census.csv', sep=',',na_values=['?'])
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297 entries, 0 to 296
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   age                        297 non-null    int64  
 1   gender                     297 non-null    object 
 2   chest pain                 297 non-null    object 
 3   rest SBP                   297 non-null    int64  
 4   cholesterol                297 non-null    int64  
 5   fasting blood sugar > 120  297 non-null    int64  
 6   rest ECG                   297 non-null    object 
 7   max HR                     297 non-null    int64  
 8   exerc ind ang              297 non-null    int64  
 9   ST by exercise             297 non-null    float64
 10  slope peak exc ST          297 non-null    object 
 11  major vessels colored      297 non-null    int64  
 12  thal                       297 non-null    object 
 13  diameter narrowing         297 non-null    int64  

In [37]:
#Definiendo las predictoras y las clases
X = datos.drop(['diameter narrowing'], axis=1)
y = datos['diameter narrowing']
X.head()
print(type(y))

<class 'pandas.core.series.Series'>


In [38]:
# get_embedding_info identifies the categorical variables and the number 
#of unique values and embedding size (half of unique values) and returns a dictionary
embedding_info = get_embedding_info(X)
embedding_info

{'gender': (2, 1),
 'chest pain': (4, 2),
 'rest ECG': (3, 2),
 'slope peak exc ST': (3, 2),
 'thal': (3, 2)}

In [39]:
# get_label_encoded_data integer encodes the categorical variables and prepares it to feed it to neural network
X_encoded,encoders = get_label_encoded_data(X)
X_encoded.head()

,age,gender,chest pain,rest SBP,cholesterol,fasting blood sugar > 120,rest ECG,max HR,exerc ind ang,ST by exercise,slope peak exc ST,major vessels colored,thal
0,63,1,3,145,233,1,1,150,0,2.3,0,0,0
1,67,1,0,160,286,0,1,108,1,1.5,1,3,1
2,67,1,0,120,229,0,1,129,1,2.6,1,2,2
3,37,1,2,130,250,0,2,187,0,3.5,0,0,1
4,41,0,1,130,204,0,1,172,0,1.4,2,0,1


In [40]:
# splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X_encoded,y)

In [41]:
# get_embeddings trains NN, extracts embeddings and return a dictionary containing the embeddings
embeddings = get_embeddings(X_train, y_train, categorical_embedding_info=embedding_info,  epochs=50,batch_size=16)

In [42]:
# if you don't like the dictionary format; convert it to dataframe for easy readibility
dfs = get_embeddings_in_dataframe(embeddings=embeddings, encoders=encoders)

In [43]:
dfs['chest pain']

,chest pain_embedding_0,chest pain_embedding_1
asymptomatic,-0.261262,-0.275426
atypical ang,0.220208,0.172708
non-anginal,0.230678,0.244894
typical ang,0.010151,-0.027243
